In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import xml.etree.ElementTree as et
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup as bs
import xmltodict
import json
import dart_fss as dart
from lxml import html
from xml.etree.ElementTree import parse
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np
import datetime
try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize
import OpenDartReader
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
kosdaq_list = pd.read_excel('kosdaq.xlsx', index_col=1)

In [3]:
crtfc_key = '본인이발급받은키입력'
def get_corpcode(crtfc_key): 
    """ 
    OpenDART 기업 고유번호 받아오기 
    return 값: 주식코드를 가진 업체의 DataFrame 
    """ 
    params = {'crtfc_key':crtfc_key} 
    items = ["corp_code","corp_name","stock_code","modify_date"] 
    item_names = ["고유번호","회사명","종목코드","수정일"] 
    url = "https://opendart.fss.or.kr/api/corpCode.xml" #요청 url
    response = requests.get(url,params=params) #url 불러오기
    zfile = ZipFile(io.BytesIO(response.content))
    fin = zfile.open(zfile.namelist()[0])  #zip file 열고
    root = et.fromstring(fin.read().decode('utf-8'))  #utf-8 디코딩

    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: # 종목코드가 있는 경우 
            data.append([]) #data에 append 
            for item in items: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns=item_names) 
    return df


stock_comp = get_corpcode(crtfc_key)

with open('고유번호.txt', 'w', encoding='utf-8') as file:
    file.write(stock_comp.to_string())


# 20200101으로 나오는 기존 date를 2020-01-01 형식으로 변형
stock_comp['수정일'] = pd.to_datetime(stock_comp['수정일'], format='%Y%m%d')

stock_comp

,고유번호,회사명,종목코드,수정일
0,00260985,한빛네트,036720,2017-06-30
1,00264529,엔플렉스,040130,2017-06-30
2,00358545,동서정보기술,055000,2017-06-30
3,00231567,애드모바일,032600,2017-06-30
4,00247939,씨모스,037600,2017-06-30
...,...,...,...,...
3711,00330044,캠시스,050110,2024-04-01
3712,00661847,화인베스틸,133820,2024-04-01
3713,00346407,큐로컴,040350,2024-04-01
3714,01428203,케이씨씨글라스,344820,2024-04-01


In [4]:
key = "본인이발급받은키입력" 
url = f"http://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={key}"
res_url = requests.get(url)

ZipFile(BytesIO(res_url.content)).extractall("./data")
xml = open("./data/CORPCODE.xml", "r", encoding='utf-8').read()
dict_type = xmltodict.parse(xml)

corp_dict = {}
for company in dict_type['result']['list']:
    if company['stock_code']:
        corp_dict[int(company['stock_code'])] = company['corp_code']
        
kosdaq_list['고유번호'] = None
for idx in kosdaq_list.index:
    kosdaq_list.loc[idx, '고유번호'] = corp_dict[idx]

In [5]:
# 기업별 고유번호 포함한 데이터프레임 저장
kosdaq_list.reset_index(inplace=True)
kosdaq_list.set_index('고유번호', drop=True, inplace=True)
kosdaq_list.to_excel('kosdaq.xlsx')
kosdaq_list

,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,주소,업종코드,설립일
고유번호,,,,,,,,,,,,,
01807738,475240,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,6088613055,서울특별시 영등포구 의사당대로 82,66199.0,20240105.0
01592049,455900,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,4358600568,"서울특별시 성동구 성수이로22길 37, 14층 1405호~1407호(성수동2가, 아...",29280.0,20170210.0
01553129,420570,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,6658600626,충청북도 청주시 청원구 오창읍 두릉유리로 50-3,20423.0,20161227.0
00262266,437730,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,6098109210,경상남도 창원시 의창구 죽전로 45,303.0,19880901.0
00260657,36220,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,2298102291,경기도 안양시 동안구 안양천동로 132,213.0,19960426.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
00165060,13030,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,NaN,NaN,NaN,NaN
00156910,19550,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,NaN,NaN,NaN,NaN
00110750,19570,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,NaN,NaN,NaN,NaN


In [34]:
kosdaq_list = pd.read_excel('kosdaq.xlsx', index_col=1)
kosdaq_list

,고유번호,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,주소,업종코드,설립일
종목코드,,,,,,,,,,,,,
475240,1807738,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,6088613055,서울특별시 영등포구 의사당대로 82,66199.0,20240105.0
455900,1592049,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,4358600568,"서울특별시 성동구 성수이로22길 37, 14층 1405호~1407호(성수동2가, 아...",29280.0,20170210.0
420570,1553129,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,6658600626,충청북도 청주시 청원구 오창읍 두릉유리로 50-3,20423.0,20161227.0
437730,262266,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,6098109210,경상남도 창원시 의창구 죽전로 45,303.0,19880901.0
36220,260657,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,2298102291,경기도 안양시 동안구 안양천동로 132,213.0,19960426.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13030,165060,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,NaN,NaN,NaN,NaN
19550,156910,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,NaN,NaN,NaN,NaN
19570,110750,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,NaN,NaN,NaN,NaN


### URL1 최대주주 성공

In [27]:
import json
import requests

key = "본인이발급받은키입력"
year = 2023
# year = 2022

# DataFrame 초기화
kosdaq_list['최대주주_성명'], kosdaq_list['최대주주_관계'], kosdaq_list['최대주주_주식종류'] = None, None, None
kosdaq_list['최대주주_기초주식수'], kosdaq_list['최대주주_기초주식지분율'] = None, None
kosdaq_list['최대주주_기말주식수'], kosdaq_list['최대주주_기말주식지분율'] = None, None

for idx in kosdaq_list.index:
    res_url_1 = requests.get(f'https://opendart.fss.or.kr/api/hyslrSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011', 
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })    
    
    json_reports = json.loads(res_url_1.text)
    
    if json_reports.get('list'):
        for company_report in json_reports['list']:
            if company_report.get('relate') == "최대주주":
                # 최대주주 정보만 가져오기
                kosdaq_list.loc[idx, '최대주주_성명'] = company_report.get('nm', None)
                kosdaq_list.loc[idx, '최대주주_관계'] = company_report.get('relate', None)
                kosdaq_list.loc[idx, '최대주주_주식종류'] = company_report.get('stock_knd', None)
                kosdaq_list.loc[idx, '최대주주_기초주식수'] = company_report.get('bsis_posesn_stock_co', None)
                kosdaq_list.loc[idx, '최대주주_기초주식지분율'] = company_report.get('bsis_posesn_stock_qota_rt', None)
                kosdaq_list.loc[idx, '최대주주_기말주식수'] = company_report.get('trmend_posesn_stock_co', None)
                kosdaq_list.loc[idx, '최대주주_기말주식지분율'] = company_report.get('trmend_posesn_stock_qota_rt', None)


In [28]:
kosdaq_list

,고유번호,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,주소,업종코드,설립일,최대주주_성명,최대주주_관계,최대주주_주식종류,최대주주_기초주식수,최대주주_기초주식지분율,최대주주_기말주식수,최대주주_기말주식지분율
종목코드,,,,,,,,,,,,,,,,,,,,
475240,1807738,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,6088613055,서울특별시 영등포구 의사당대로 82,66199.0,20240105.0,None,None,None,None,None,None,None
455900,1592049,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,4358600568,"서울특별시 성동구 성수이로22길 37, 14층 1405호~1407호(성수동2가, 아...",29280.0,20170210.0,None,None,None,None,None,None,None
420570,1553129,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,6658600626,충청북도 청주시 청원구 오창읍 두릉유리로 50-3,20423.0,20161227.0,None,None,None,None,None,None,None
437730,262266,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,6098109210,경상남도 창원시 의창구 죽전로 45,303.0,19880901.0,None,None,None,None,None,None,None
36220,260657,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,2298102291,경기도 안양시 동안구 안양천동로 132,213.0,19960426.0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13030,165060,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None
19550,156910,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None
19570,110750,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None


### URL2 소액주주 성공

In [30]:
import json
import requests

key = "본인이발급받은키입력"
year = 2023
# year = 2022

kosdaq_list['소액주주수'], kosdaq_list['주주수'], kosdaq_list['소액주주비율'] = None, None, None
kosdaq_list['소액주주_주식수'], kosdaq_list['총주식수'], kosdaq_list['소액주주_주식지분율'] = None, None, None

for idx in kosdaq_list.index:
    res_url_2 = requests.get(f'https://opendart.fss.or.kr/api/mrhlSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011',  
                                 headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })   
    
    json_reports = json.loads(res_url_2.text)
    if json_reports.get('list'):
        for company_report in json_reports['list']:
            kosdaq_list.loc[idx, '소액주주수'] = company_report.get('shrholdr_co', None)
            kosdaq_list.loc[idx, '주주수'] = company_report.get('shrholdr_tot_co', None)
            kosdaq_list.loc[idx, '소액주주비율'] = company_report.get('shrholdr_rate', None)
            kosdaq_list.loc[idx, '소액주주_주식수'] = company_report.get('hold_stock_co', None)
            kosdaq_list.loc[idx, '총주식수'] = company_report.get('stock_tot_co', None)
            kosdaq_list.loc[idx, '소액주주_주식지분율'] = company_report.get('hold_stock_rate', None)


In [32]:
kosdaq_list

,고유번호,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,...,최대주주_기초주식수,최대주주_기초주식지분율,최대주주_기말주식수,최대주주_기말주식지분율,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율
종목코드,,,,,,,,,,,,,,,,,,,,,
475240,1807738,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,6088613055,...,None,None,None,None,None,None,None,None,None,None
455900,1592049,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,4358600568,...,None,None,None,None,None,None,None,None,None,None
420570,1553129,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,6658600626,...,None,None,None,None,None,None,None,None,None,None
437730,262266,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,6098109210,...,None,None,None,None,None,None,None,None,None,None
36220,260657,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,2298102291,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13030,165060,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,NaN,...,None,None,None,None,None,None,None,None,None,None
19550,156910,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,NaN,...,None,None,None,None,None,None,None,None,None,None
19570,110750,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,NaN,...,None,None,None,None,None,None,None,None,None,None


In [33]:
kosdaq_list['소액주주수'].unique()

array([None], dtype=object)

### URL3 감사의견 성공

In [23]:
import json
import requests

key = "본인이발급받은키입력"
year = 2023
# year = 2022

# DataFrame 초기화
kosdaq_list['감사의견'], kosdaq_list['감사의견_강조사항'], kosdaq_list['감사의견_핵심감사사항'] = None, None, None

for idx in kosdaq_list.index:
    res_url_3 = requests.get(f'https://opendart.fss.or.kr/api/accnutAdtorNmNdAdtOpinion.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011', headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })    
    
    json_reports = json.loads(res_url_3.text)
    
    if json_reports.get('list'):
        adt_opinion_list, emphs_matter_list, core_adt_matter_list = [], [], []
        for company_report in json_reports['list']:
            adt_opinion_list.append(company_report.get('adt_opinion', ''))
            emphs_matter_list.append(company_report.get('emphs_matter', ''))
            core_adt_matter_list.append(company_report.get('core_adt_matter', ''))

        # 리스트에 있는 값을 하나로 합쳐서 DataFrame에 할당
        kosdaq_list.loc[idx, '감사의견'] = ' '.join(adt_opinion_list)
        kosdaq_list.loc[idx, '감사의견_강조사항'] = ' '.join(emphs_matter_list)
        kosdaq_list.loc[idx, '감사의견_핵심감사사항'] = ' '.join(core_adt_matter_list)


In [24]:
kosdaq_list

,고유번호,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,...,최대주주_기말주식지분율,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율,감사의견,감사의견_강조사항,감사의견_핵심감사사항
종목코드,,,,,,,,,,,,,,,,,,,,,
475240,1807738,하나32호스팩,금융 지원 서비스업,기업인수합병,45378,12월,박종찬,NaN,서울특별시,6088613055,...,None,None,None,None,None,None,None,None,None,None
455900,1592049,엔젤로보틱스,특수 목적용 기계 제조업,"재활 전문 웨어러블 로봇, 근력 보조 무동력 웨어러블 슈트, 일상생활 보조 웨어러블...",45377,12월,공경철,http://www.angel-robotics.com/ko/,서울특별시,4358600568,...,None,None,None,None,None,None,None,None,None,None
420570,1553129,제이투케이바이오,기타 화학제품 제조업,화장품 원료(천연소재 바이오소재 등),45376,12월,이재섭,http://www.j2kbio.com,충청북도,6658600626,...,None,None,None,None,None,None,None,None,None,None
437730,262266,삼현,자동차 신품 부품 제조업,"스마트 액추에이터, 스마트 파워유닛",45372,12월,황성호,http://www.samhyun.co.kr,경상남도,6098109210,...,None,None,None,None,None,None,None,None,None,None
36220,260657,오상헬스케어,의료용품 및 기타 의약 관련제품 제조업,"생화학 측정기 및 센서, 면역진단기기 및 마커, 분자진단기기 및 시약 등",45364,12월,홍승억,http://www.osanghc.com,경기도,2298102291,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13030,165060,하이록코리아,1차 철강 제조업,관이음쇠,32857,12월,"문휴건, 문창환 각자대표이사",http://www.hy-lok.com,부산광역시,NaN,...,None,None,None,None,None,None,None,None,None,None
19550,156910,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,32757,12월,공동대표이사 안재광공동대표이사 소우 에이이치로,http://www.sbik.co.kr,서울특별시,NaN,...,None,None,None,None,None,None,None,None,None,None
19570,110750,리더스 기술투자,신탁업 및 집합투자업,금융투자,32581,12월,엄원진,http://www.geminivc.co.kr,서울특별시,NaN,...,None,None,None,None,None,None,None,None,None,None


In [25]:
kosdaq_list.to_excel('kosdaq_report_2020.xlsx')
kosdaq_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1718 entries, 475240 to 6920
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   고유번호          1718 non-null   int64  
 1   회사명           1718 non-null   object 
 2   업종            1718 non-null   object 
 3   주요제품          1715 non-null   object 
 4   상장일           1718 non-null   int64  
 5   결산월           1718 non-null   object 
 6   대표자명          1718 non-null   object 
 7   홈페이지          1572 non-null   object 
 8   지역            1718 non-null   object 
 9   사업자등록번호       690 non-null    object 
 10  주소            690 non-null    object 
 11  업종코드          690 non-null    float64
 12  설립일           690 non-null    float64
 13  최대주주_성명       0 non-null      object 
 14  최대주주_관계       0 non-null      object 
 15  최대주주_주식종류     0 non-null      object 
 16  최대주주_기초주식수    0 non-null      object 
 17  최대주주_기초주식지분율  0 non-null      object 
 18  최대주주_기말주식수    0 non-null    